Importante 

Tener en cuenta bajadas de Procedimientos, Personas Armas, Divisas , vehiculos (secuestrados, ministerio), Narcotrafico general,Objetos y pegar en la carpeta bajadas de fecha del primero del 1 del mes en informar hasta fecha del informe 

1 -importo las librerias

In [11]:
import pandas as pd
import re
import numpy as np
from openpyxl import load_workbook

2-Coloco los parametros a tener en cuenta para donde se encuentra los archivos de bajada


In [12]:
FECHA_INICIO = '01/10/2024'
FECHA_FINAL = '14/10/2024'
ESTADO = "DISPONIBLE ESTADISTICA"

LUGARES_CATALOGADOS= {
                    "SECTOR RESTRINGIDO":	"SECTOR DE SEGURIDAD RESTRINGIDA AEROPORTUARIA",
                    "SECTOR PUBLICO"	:"INSTALACIONES DE ACCESO AL PUBLICO GENERAL",
                    "PERIMETRO DE JURISDICCION":	"PERIMETRO AEROPORTUARIO",
                    "FUERA DE JURISDICCION":	"DESPLIEGUE",
                    "TRANSPORTE PUBLICO": "TRANSPORTE PUBLICO",
                    "S/D": "S/D"
}

UNIDADES_MUNICIPIOS = {
    "EZE": "JOSÉ M. EZEIZA",
    "AER": "COMUNA 14",
    "SFO": "SAN FERNANDO",
    "BHI": "BAHÍA BLANCA",
    "MDP": "GENERAL PUEYRREDÓN",
    "MDZ": "LAS HERAS",
    "COR": "CAPITAL",
    "JUA": "9 DE JULIO",
    "LUI": "JUAN MARTÍN DE PUEYRREDÓN",
    "MLG": "MALARGÜE",
    "RAF": "SAN RAFAEL",
    "RCU": "RÍO CUARTO",
    "SRO": "CAPITAL",
    "JUJ": "EL CARMEN",
    "SAL": "LA CAPITAL",
    "CAT": "VALLE VIEJO",
    "LAR": "CAPITAL",
    "SGO": "CAPITAL",
    "TRH": "RÍO HONDO",
    "TUC": "CRUZ ALTA",
    "IGU": "IGUAZÚ",
    "FSA": "FORMOSA",
    "ROS": "ROSARIO",
    "RCQ": "GENERAL OBLIGADO",
    "POS": "CAPITAL",
    "CRR": "CAPITAL",
    "LIB": "PASO DE LOS LIBRES",
    "PAR": "PARANÁ",
    "RES": "SAN FERNANDO",
    "SVO": "LA CAPITAL",
    "BAR": "BARILOCHE",
    "TRE": "RAWSON",
    "NEU": "CONFLUENCIA",
    "CAL": "LAGO ARGENTINO",
    "CHP": "LÁCAR",
    "CRV": "ESCALANTE",
    "PMY": "BIEDMA",
    "ESQ": "FUTALEUFÚ",
    "GAL": "GÜER AIKE",
    "GDE": "RÍO GRANDE",
    "USU": "USHUAIA",
    "VIE": "ADOLFO ALSINA"
}

PROVINCIAS = {
    "C.A. BUENOS AIRES": "CIUDAD AUTONOMA DE BUENOS AIRES",
    "BUENOS AIRES": "BUENOS AIRES",
    "SANTA CRUZ": "SANTA CRUZ",
    "SANTA FE": "SANTA FE",
    "MENDOZA": "MENDOZA",
    "TUCUMÁN": "TUCUMAN",
    "TIERRA DEL FUEGO": "TIERRA DEL FUEGO ANTARTIDA E ISLAS DEL ATLANTICO SUR",
    "SALTA": "SALTA",
    "NEUQUÉN": "NEUQUEN",
    "CÓRDOBA": "CORDOBA",
    "LA PAMPA": "LA PAMPA",
    "RÍO NEGRO": "RIO NEGRO",
    "CATAMARCA": "CATAMARCA",
    "MISIONES": "MISIONES",
    "SAN JUAN": "SAN JUAN",
    "CORRIENTES": "CORRIENTES",
    "CHUBUT": "CHUBUT",
    "FORMOSA": "FORMOSA",
    "JUJUY": "JUJUY",
    "SANTIAGO DEL ESTERO": "SANTIAGO DEL ESTERO",
    "LA RIOJA": "LA RIOJA",
    "CHACO": "CHACO",
    "ENTRE RÍOS": "ENTRE RIOS",
    "SAN LUIS": "SAN LUIS"
}



hoja_procedimientos  = pd.DataFrame()
hoja_vehiculos_personas  = pd.DataFrame()
hoja_personal_elementos  = pd.DataFrame()
hoja_detenidos_aprendidos  = pd.DataFrame()
hoja_trafico_personas  = pd.DataFrame()
hoja_otros_delitos  = pd.DataFrame()
hoja_otros_eventos  = pd.DataFrame()
hoja_otros_fallecidos  = pd.DataFrame()
hoja_otros_abatidos  = pd.DataFrame()


excel_base_informada = pd.DataFrame()

3- Transformo los excel de las bajadas de los dataframe para poder procesarlos en posterior

In [13]:
excel_bajada_procedimientos = pd.read_excel('bajadas/bajada_general.xls')
excel_bajada_arma = pd.read_excel('bajadas/bajada_arma.xls')
excel_bajada_vehiculos_secuestros = pd.read_excel('bajadas/bajada_vehiculos_secuestrados.xls')
excel_bajada_vehiculos_secuestros_ministerio = pd.read_excel('bajadas/bajada_ministerio_vehiculos.xls')
excel_bajada_personas = pd.read_excel('bajadas/bajada_general_persona.xls')
excel_bajada_divisas = pd.read_excel('bajadas/bajada_divisas_secuestradas.xls')
excel_bajada_narcotrafico = pd.read_excel('bajadas/bajada_general_narcotrafico.xls')
excel_bajada_operaciones = pd.read_excel('bajadas/operaciones.xlsx')

Leer base de informado de datos leer la base de datos "data/Base_informada" y


In [14]:
# Cargar el archivo Excel con openpyxl

excel_base_informada = load_workbook("data/Base_informada.xlsx")
hoja = excel_base_informada['GEOG. PROCEDIMIENTO']

# Crear una lista para almacenar los datos
data = []

# Obtener los títulos de las columnas (fila 3, desde la columna B)
titulos = [hoja.cell(row=3, column=col).value for col in range(2, hoja.max_column + 1)]

# Leer los datos desde la fila 4 en adelante, empezando en la columna B
for row in hoja.iter_rows(min_row=4, min_col=2, max_col=hoja.max_column, values_only=True):
    data.append(row)

# Crear un DataFrame a partir de los datos
excel_base_informada = pd.DataFrame(data, columns=titulos)



Filtrar solo que esta que estan disponible en estadistica y las fechas estipuladas de información 

In [15]:
# Convertir las fechas de inicio y final a formato datetime
fecha_inicio = pd.to_datetime(FECHA_INICIO, format='%d/%m/%Y')
fecha_final = pd.to_datetime(FECHA_FINAL, format='%d/%m/%Y')

# Filtrar las filas donde el estado del parte es 'DISPONIBLE ESTADISTICA' y la fecha esté en el rango deseado
excel_bajada_procedimientos = excel_bajada_procedimientos[
    (excel_bajada_procedimientos['ESTADO_PARTE'] == 'DISPONIBLE ESTADISTICA') &
    (excel_bajada_procedimientos['DENUNCIAFECHA'] >= fecha_inicio) &
    (excel_bajada_procedimientos['DENUNCIAFECHA'] <= fecha_final)
].copy()





Funciones para procesar cada celda y columna del informe 

In [16]:
def procesar_causa_judicial(row):
    causa = row['CAUSAJUDICIALNUMERO']
    if pd.isna(causa) or causa in ["", "S/D", "A/S", "N/C"]:
        return f"{row['TIPO_CAUSA_INTERNA']} {row['CAUSA_INTERNA_NUMERO']}".strip()
    
    causa_str = str(causa).strip()
    
    prefijos = ["NRO.", "N°", "EXPT", "EXPEDIENTE", "CAUSA"]
    
    for prefijo in prefijos:
        if causa_str.upper().startswith(prefijo):
            causa_str = causa_str[len(prefijo):].strip()
    
    # Eliminar cualquier carácter que no sea número, letra o /ML
    causa_str = re.sub(r'[^\w/]', '', causa_str)
    
    return causa_str

contador_global_sigipol = {}
def generar_codigo_sigipol(row):
    id_operativo = row['ID_OPERATIVO']
    if id_operativo not in contador_global_sigipol:
        contador_global_sigipol[id_operativo] = 0
    contador_global_sigipol[id_operativo] += 1
    contador = contador_global_sigipol[id_operativo]
    return id_operativo + " (" + str(contador) + ")"


contador_global_oper = {}
def generar_codigo_oper(row):
    id_operativo = row['ID_OPERATIVO']
    if id_operativo not in contador_global_oper:
        contador_global_oper[id_operativo] = 0
    contador_global_oper[id_operativo] += 1
    contador = contador_global_oper[id_operativo]
    return id_operativo + " (" + str(contador) + ")"

    

def procesar_descripcion(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if tipo == "DENUNCIA":
        return "DENUNCIA POLICIAL"
    elif tipo == "CONTROL PREVENTIVO":
        return f"CONTROL PREVENTIVO - {procesar_lugar(row)}"
    elif tipo == "ORDEN DE ALLANAMIENTO":
        return "ORDEN DE ALLANAMIENTO"
    elif tipo == "ORDEN DE ALLANAMIENTO / DETENCIÓN":
        return "ORDEN DE ALLANAMIENTO"
    else:
        return "OTRO MANDATO JUDICIAL"


def procesar_tipo(row):
    tipo = row['TIPO_PROCEDIMIENTO']
    if pd.isna(tipo):
        return ""
    elif tipo == "DENUNCIA"  or tipo == "CONTROL PREVENTIVO" :
        return "ORDEN POLICIAL"
    else:
        return "ORDEN JUDICIAL"
    
def procesar_provincia(row):
    provincia = row['PROVINCIA']
    if pd.isna(provincia):
        return ""
    return PROVINCIAS.get(provincia, provincia)
    
def procesar_municipio(row):
    unidad = row['UOSP']
    if pd.isna(unidad):
        return ""
    return UNIDADES_MUNICIPIOS.get(unidad, unidad)

def procesar_lugar(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    return  LUGARES_CATALOGADOS[lugar]

def procesar_direccion(row):
    lugar = row['LUGAR_CATALOGADO_NIVEL_1']
    if lugar == "FUERA DE JURISDICCION":
        return str(row['CALLE']) + " " + str(row['NUMERO']) + ", " + str(row['CIUDAD']) + " - " + str(row['PARTIDO'])
    else:
        return ""



Ejucucion de funciones 

In [17]:
hoja_procedimientos['ID_OPERATIVO'] = excel_bajada_procedimientos.apply(procesar_causa_judicial, axis=1)
hoja_procedimientos['ID_PROCEDIMIENTO'] = hoja_procedimientos.apply(generar_codigo_sigipol, axis=1)
hoja_procedimientos['UNIDAD_INTERVINIENTE'] = excel_bajada_procedimientos['UOSP']
hoja_procedimientos['DESCRIPCIÓN'] = excel_bajada_procedimientos.apply(procesar_descripcion, axis=1)
hoja_procedimientos['TIPO_INTERVENCION'] = excel_bajada_procedimientos.apply(procesar_tipo, axis=1)
hoja_procedimientos['PROVINCIA'] = excel_bajada_procedimientos.apply(procesar_provincia, axis=1)
hoja_procedimientos['DEPARTAMENTO O PARTIDO'] = excel_bajada_procedimientos.apply(procesar_municipio, axis=1)
hoja_procedimientos['LOCALIDAD'] = "-"
hoja_procedimientos['DIRECCION'] = excel_bajada_procedimientos.apply(procesar_direccion, axis=1)
hoja_procedimientos['LATITUD'] = excel_bajada_procedimientos['GEOREFERENCIA_Y']
hoja_procedimientos['LONGITUD'] = excel_bajada_procedimientos['GEOREFERENCIA_X']
hoja_procedimientos['FECHA'] = pd.to_datetime(excel_bajada_procedimientos['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
hoja_procedimientos['HORA'] = pd.to_datetime(excel_bajada_procedimientos['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')



hoja_procedimientos['FUERZA_INTERVINIENTE'] = "PSA"
hoja_procedimientos['ZONA_SEGURIDAD_FRONTERAS'] = "-"
hoja_procedimientos['PASO_FRONTERIZO'] = "-"
hoja_procedimientos['OTRAS AGENCIAS INTERVINIENTES'] = "-"
hoja_procedimientos['Observaciones - Detalles'] = "-"


Filtrar con la ya informado es decir  filtra con lo nuevo de procedimientos

In [18]:


hoja_procedimientos = hoja_procedimientos[
    ~hoja_procedimientos['ID_PROCEDIMIENTO'].isin(excel_base_informada['ID_PROCEDIMIENTO'])
].copy()



Acomdamiento de la informacion como de la hoja de procedimientos 

In [19]:

hoja_procedimientos = hoja_procedimientos[['FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]


nombre_archivo_modelo = "Planilla_modelo.xlsx"
nombre_archivo_informe = "Planilla_informe.xlsx"
wb = load_workbook(nombre_archivo_modelo)
ws = wb['GEOG. PROCEDIMIENTO']

# Escribir los datos del DataFrame desde la fila 4
for row_num, row in enumerate(hoja_procedimientos.itertuples(index=False), start=4):
    for col_num, value in enumerate(row, start=2):  # Comienza desde la columna B (número 2)
        ws.cell(row=row_num, column=col_num).value = value

# Guardar los cambios
wb.save(nombre_archivo_informe)

print(f"\nArchivo Excel '{nombre_archivo_informe}' modificado con éxito.")


Archivo Excel 'Planilla_informe.xlsx' modificado con éxito.


Control de salidad de la informacion

In [20]:
# Aplicar la función procesar_causa_judicial a la copia

# # Contar cuántos casos se procesaron vs. cuántos se mantuvieron igual
# procesados = (wb_nueva_hoja['CAUSAJUDICIALNUMERO'] != wb_nueva_hoja['ID_OPERATIVO']).sum()
# print(f"\nCasos procesados: {procesados}")
# print(f"Casos sin cambios: {len(wb_nueva_hoja) - procesados}")


